In [40]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelBinarizer, KBinsDiscretizer, OneHotEncoder, OrdinalEncoder, MultiLabelBinarizer, MinMaxScaler, FunctionTransformer, StandardScaler, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
import warnings
warnings.filterwarnings('ignore')
from sklearn import __version__
__version__

'1.5.1'

# EDA

In [ ]:
# prompt: code to upload file in pandas

from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Assuming the uploaded file is a CSV, you can load it into a pandas DataFrame
import io
df = pd.read_csv(io.BytesIO(uploaded[fn]))
# Replace 'fn' with the actual file name if you have only one file uploaded

# Now you can work with the DataFrame 'df'
df.head()

Saving train.csv to train (1).csv
User uploaded file "train (1).csv" with length 4684734 bytes


,Location,Cross_Street,Latitude,Longitude,Date_Reported,Date_Occurred,Time_Occurred,Area_ID,Area_Name,Reporting_District_no,...,Victim_Age,Victim_Sex,Victim_Descent,Premise_Code,Premise_Description,Weapon_Used_Code,Weapon_Description,Status,Status_Description,Crime_Category
0,4500 CARPENTER AV,NaN,34.1522,-118.3910,03/09/2020 12:00:00 AM,03/06/2020 12:00:00 AM,1800.0,15.0,N Hollywood,1563.0,...,75.0,M,W,101.0,STREET,NaN,NaN,IC,Invest Cont,Property Crimes
1,45TH ST,ALAMEDA ST,34.0028,-118.2391,02/27/2020 12:00:00 AM,02/27/2020 12:00:00 AM,1345.0,13.0,Newton,1367.0,...,41.0,M,H,216.0,SWAP MEET,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,Property Crimes
2,600 E MARTIN LUTHER KING JR BL,NaN,34.0111,-118.2653,08/21/2020 12:00:00 AM,08/21/2020 12:00:00 AM,605.0,13.0,Newton,1343.0,...,67.0,M,B,501.0,SINGLE FAMILY DWELLING,NaN,NaN,IC,Invest Cont,Property Crimes
3,14900 ORO GRANDE ST,NaN,34.2953,-118.4590,11/08/2020 12:00:00 AM,11/06/2020 12:00:00 AM,1800.0,19.0,Mission,1924.0,...,61.0,M,H,101.0,STREET,NaN,NaN,IC,Invest Cont,Property Crimes
4,7100 S VERMONT AV,NaN,33.9787,-118.2918,02/25/2020 12:00:00 AM,02/25/2020 12:00:00 AM,1130.0,12.0,77th Street,1245.0,...,0.0,X,X,401.0,MINI-MART,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,Property Crimes


In [ ]:
# plt.hist(df.Reporting_District_no, bins=100)
# df.info()

In [ ]:
# df.describe()

### for local runtime

In [3]:
!ls

crime-cast-forecasting-crime-categories.zip  data  main.ipynb


In [4]:
df = pd.read_csv('data/train.csv')

In [22]:
len(df.columns)

22

# Preprocessing

#### Spliting train and test sets

In [5]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=1)

In [7]:
X_train.shape, X_test.shape

((16000, 21), (4000, 21))

In [8]:
class CustomModification(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.mlb = MultiLabelBinarizer()
        modus_operandi = X['Modus_Operandi'].map(lambda x: () if type(x) == float else tuple(map(int,x.split(' '))))
        self.mlb.fit(modus_operandi)
        return self

    def transform(self, X, y=None):
        X = X.copy()
        self.clean(X)

        X['reported_year'] = X['Date_Reported'].dt.year
        X['reported_month'] = X['Date_Reported'].dt.month
        X['reported_day'] = X['Date_Reported'].dt.day

        X['occurred_year'] = X['Date_Occurred'].dt.year
        X['occurred_month'] = X['Date_Occurred'].dt.month
        X['occurred_day'] = X['Date_Occurred'].dt.day

        MO = self.mlb.transform(X['Modus_Operandi'])
        X2 = pd.DataFrame(MO, index=X.index)
        X2.columns = X2.columns.map(lambda x: 'MO_' + str(x))
        X = pd.concat([X, X2], axis=1)

        return X

    def clean(self, X):
      X["Latitude"] = X['Latitude'].map(lambda x: np.nan if x == 0 else x)
      X["Longitude"] = X['Longitude'].map(lambda x: np.nan if x == 0 else x)
      X['Date_Reported'] = pd.to_datetime(X['Date_Reported'], format='%m/%d/%Y %H:%M:%S %p')
      X['Date_Occurred'] = pd.to_datetime(X['Date_Occurred'], format='%m/%d/%Y %H:%M:%S %p')
      X.loc[X['Victim_Age'] < 0, 'Victim_Age'] = 0
      X["Modus_Operandi"] = X['Modus_Operandi'].map(lambda x: () if type(x) == float else tuple(map(int,x.split(' '))))



In [9]:
cat_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist')),
])
num_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('encoder', KBinsDiscretizer(strategy='kmeans', encode='onehot-dense')),
])

In [26]:
pre_proc = ColumnTransformer([
  ('dropper', 'drop', [0, 1, 4, 5, 8, 10, 11, 16, 18, 20]),
  ('ofe', OrdinalEncoder(), [10]),
  ('ss', StandardScaler(with_mean=True, with_std=True), [6, 12]),
  ('cp1', cat_pipe, [13, 14]),
  ('mms', MinMaxScaler(), list(range(21, 27))),
  ('loc', num_pipe, [2, 3]),
  ('uofe', OneHotEncoder(sparse_output=False, handle_unknown='infrequent_if_exist', min_frequency=0.05), [7,9,15,17,19]),
], remainder='passthrough').set_output(transform='pandas')

In [27]:
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.fit_transform(y_test)

In [28]:
lb = LabelBinarizer()
y_train_bin = lb.fit_transform(y_train)
y_test_bin = lb.fit_transform(y_test)

In [29]:
pipe1 = make_pipeline(CustomModification(), pre_proc, GaussianNB())

In [30]:
pipe2 = make_pipeline(CustomModification(), pre_proc, DecisionTreeClassifier())

In [31]:
pipe3 = make_pipeline(CustomModification(), pre_proc, RandomForestClassifier(random_state=0))

In [32]:
pipe4 = make_pipeline(CustomModification(), pre_proc, LogisticRegression(solver='liblinear',penalty='l1'))
# pipe4 = make_pipeline(CustomModification(), pre_proc, LogisticRegression())

In [41]:
pipe1.fit(X_train, y_train_enc)
print('Training Score: ', pipe1.score(X_train, y_train_enc))
print('Testing Score: ', pipe1.score(X_test, y_test_enc))

Training Score:  0.3340625
Testing Score:  0.312


In [42]:
print('Decision Tree Classifier')
print('Label Encode Ordinaly')
pipe2.fit(X_train, y_train_enc)
print('Training Score: ', pipe2.score(X_train, y_train_enc))
print('Testing Score: ', pipe2.score(X_test, y_test_enc))
print('Label Binarized')
pipe2.fit(X_train, y_train_bin)
print('Training Score: ', pipe2.score(X_train, y_train_bin))
print('Testing Score: ', pipe2.score(X_test, y_test_bin))

Decision Tree Classifier
Label Encode Ordinaly
Training Score:  1.0
Testing Score:  0.92525
Label Binarized
Training Score:  1.0
Testing Score:  0.92425


In [43]:
print('Random Forest Classifier')
print('Label Encode Ordinaly')
pipe3.fit(X_train, y_train_enc)
print('Training Score: ', pipe3.score(X_train, y_train_enc))
print('Testing Score: ', pipe3.score(X_test, y_test_enc))
print('Label Binarized')
pipe3.fit(X_train, y_train_bin)
print('Training Score: ', pipe3.score(X_train, y_train_bin))
print('Testing Score: ', pipe3.score(X_test, y_test_bin))

Random Forest Classifier
Label Encode Ordinaly
Training Score:  1.0
Testing Score:  0.94625
Label Binarized
Training Score:  1.0
Testing Score:  0.9205


In [44]:
pipe4.fit(X_train, y_train_enc)
print('Training Score: ', pipe4.score(X_train, y_train_enc))
print('Testing Score: ', pipe4.score(X_test, y_test_enc))

Training Score:  0.9555
Testing Score:  0.951


In [45]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
cvl = cross_val_score(pipe4, X, y, cv=skf, scoring='accuracy')
cvl.mean()

0.9496

In [46]:
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedKFold
pipe = Pipeline([
    ('custom',  ColumnTransformer()),
    ('pre_process', pre_proc),
    ('classifier', BaggingClassifier(DecisionTreeClassifier()))
])

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1, test_size=0.2)

param_grid = {
    'classifier__max_samples': [0.2, 0.6, 1.0],
    'classifier__max_features': [2, 5, 0.5, 1.0],
    'classifier__bootstrap': [True, False],
    'classifier__bootstrap_features': [True, False],
    'classifier__n_estimators': [10, 20, 50],
}

grid_search = GridSearchCV(pipe, param_grid, cv=skf, scoring='accuracy')
grid_search.fit(X, y)

print("Best Parameters: ", grid_search.best_params_)

SyntaxError: ':' expected after dictionary key (422202046.py, line 14)

# Final

In [ ]:
pipe4.fit(X, y)

Pipeline(steps=[('custommodification', CustomModification()),
                ('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('dropper', 'drop',
                                                  [0, 1, 4, 5, 8, 10, 11, 16,
                                                   18, 20]),
                                                 ('ofe', OrdinalEncoder(),
                                                  [10]),
                                                 ('ss', StandardScaler(),
                                                  [6, 12]),
                                                 ('cp1',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle...
                                                 ('mms', MinMaxScaler(),
                                                  [21, 22, 23, 24, 25, 26]),
                                                 ('loc',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('encoder',
                                                                   KBinsDiscretizer(encode='onehot-dense',
                                                                                    strategy='kmeans'))]),
                                                  [2, 3]),
                                                 ('uofe',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                min_frequency=0.5,
                                                                sparse_output=False),
                                                  [7, 9, 15, 17, 19])])),
                ('logisticregression',
                 LogisticRegression(penalty='l1', solver='liblinear'))])

In [ ]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
import io
Xt = pd.read_csv(io.BytesIO(uploaded[fn]))

Saving test.csv to test (1).csv
User uploaded file "test (1).csv" with length 1082723 bytes


In [ ]:
prediction = pipe4.predict(Xt)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) [1011, 1231, 1255, 1314, 2026, 2043, 2044, 218, 2208, 2308, 3030, 306, 320, 393, 403, 518, 540, 854] will be ignored
  warnings.warn(


In [ ]:
# sub = pd.DataFrame(prediction, index=Xt.index, columns=['Crime_Category'])
sub = pd.DataFrame()
sub['Crime_Category'] = prediction
sub.index += 1
sub.index.name = 'ID'
# sub['Crime_Category'] = prediction

In [ ]:
# prompt: write code to download sub

sub.to_csv('submission.csv')
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>